<a href="https://colab.research.google.com/github/hchaparov/Dynamic_Pricing_MARL/blob/main/SA_Pong_PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "stable-baselines3[extra]>=2.0.0a4"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 24.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.

In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder
from IPython.display import HTML
from base64 import b64encode
import gymnasium as gym

In [3]:
env_id = "PongNoFrameskip-v4"
# Make the Atari environment with single frame
env = make_atari_env(env_id, n_envs=8, seed=0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
model = PPO("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=200000)

Using cpu device
Wrapping the env in a VecTransposeImage.
Saving video to /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-13417-to-step-14417.mp4
Moviepy - Building video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-13417-to-step-14417.mp4.
Moviepy - Writing video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-13417-to-step-14417.mp4



Moviepy - Done !
Moviepy - video ready /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-13417-to-step-14417.mp4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3.72e+03 |
|    ep_rew_mean     | -20.4    |
| time/              |          |
|    fps             | 403      |
|    iterations      | 1        |
|    time_elapsed    | 40       |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.7e+03     |
|    ep_rew_mean          | -20.4       |
| time/                   |             |
|    fps                  | 97          |
|    iterations           | 2           |
|    time_elapsed         | 337         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.008402012 |
|    clip_fraction        | 0.042       |
|    clip_range           | 0.2   

In [6]:
!pip install ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=5415eff9ff71d3390fcb967a1d8193b03fd7c832f3f3a4d9d40371b7514dc4ef
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [14]:
video_folder = "logs/videos/"
video_length = 1000# Record the video

env = VecVideoRecorder(env, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix=f"trained-agent-{env_id}")

# Reset the environment
obs = env.reset()

# Run the environment for the specified number of steps using the trained model
for _ in range(video_length + 1):
    action, _ = model.predict(obs)  # Predict the action using the trained model
    obs, _, _, _ = env.step(action)  # Take the action in the environment

# Close the VecVideoRecorder and save the video
env.close()

# Function to display the video
def show_video(video_path):
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'<video width="480" height="320" controls><source src="{data_url}" type="video/mp4"></video>')

# Display the video
show_video(video_folder + f"trained-agent-{env_id}-step-0-to-step-{video_length}.mp4")

Saving video to /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-69714-to-step-70714.mp4
Moviepy - Building video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-69714-to-step-70714.mp4.
Moviepy - Writing video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-69714-to-step-70714.mp4



Moviepy - Done !
Moviepy - video ready /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-69714-to-step-70714.mp4
Saving video to /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-1001-to-step-2001.mp4
Moviepy - Building video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-1001-to-step-2001.mp4.
Moviepy - Writing video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-1001-to-step-2001.mp4



Moviepy - Done !
Moviepy - video ready /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-1001-to-step-2001.mp4
Saving video to /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-0-to-step-1000.mp4
Moviepy - Building video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-0-to-step-1000.mp4.
Moviepy - Writing video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-0-to-step-1000.mp4


In [15]:
model.save("Sa_pong_ppo_model")

In [13]:
model.learn(total_timesteps=100000)

Saving video to /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-55378-to-step-56378.mp4
Moviepy - Building video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-55378-to-step-56378.mp4.
Moviepy - Writing video /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-55378-to-step-56378.mp4



Moviepy - Done !
Moviepy - video ready /content/logs/videos/trained-agent-PongNoFrameskip-v4-step-55378-to-step-56378.mp4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.26e+04 |
|    ep_rew_mean     | 3.2      |
| time/              |          |
|    fps             | 400      |
|    iterations      | 1        |
|    time_elapsed    | 40       |
|    total_timesteps | 16384    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.17e+04   |
|    ep_rew_mean          | 5.85       |
| time/                   |            |
|    fps                  | 104        |
|    iterations           | 2          |
|    time_elapsed         | 312        |
|    total_timesteps      | 32768      |
| train/                  |            |
|    approx_kl            | 0.21535844 |
|    clip_fraction        | 0.558      |
|    clip_range           | 0.2        |
|    